# Scraping

## Importing libraries

In [1]:
import pandas as pd 
import numpy as np
import datetime
import pytz

## Defning the dataset that has been scraped by Scraping python script

In [3]:
df = pd.read_csv('Tesla_Comments_filtered.csv', delimiter=";")


C:\Users\maart\AppData\Local\Temp\ipykernel_9600\369132780.py:1: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Tesla_Comments_filtered.csv', delimiter=";")


In [4]:
df = df[['subreddit','id','body','created_utc','parent_id']]


## Numerical transformation

In [5]:
# Convert "body" column to integers
df['created_utc'] = pd.to_numeric(df['created_utc'], errors='coerce')
df = df.dropna(subset=['created_utc'])

# Define function to convert Unix timestamp to datetime object
def unix_to_datetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp)

# Apply function to Unix timestamp column
df['date'] = df['created_utc'].apply(unix_to_datetime)

In [6]:
print(df['body'].isnull().sum())

20149


In [7]:
df.dropna(inplace=True)

## Usage of pysentimiento to create sentiment scores

In [ ]:
from pysentimiento import create_analyzer
emotion_analyzer = create_analyzer(task="sentiment", lang="en")

In [ ]:
# Be aware applying this model takes approx two hours to run
#Using the emotion analyer on all reddit comments and put it under a new column 
df['emotion'] = df['body'].apply(emotion_analyzer.predict)


In [ ]:
#This was done to list the probabilities of the analyzeroutput
df['probas'] = df['emotion'].apply(lambda x: list(x.probas.items()))


## Calculating the sentiment scores as one for every reddit comment

In [11]:
#This was done to get a score that can be used in the next step to match frequencies with stock prices

calc_reg_output = lambda row: -1 * row['probas'][0][1] + 0 * row['probas'][1][1] + 1 * row['probas'][2][1]


In [12]:
#Now we apply the above function to every single row so that we have one score for every comment
df['reg_output'] = df.apply(calc_reg_output, axis=1)


KeyError: 'probas'

In [ ]:
df

,subreddit,id,body,created_utc,parent_id,date,emotion,probas,reg_output
0,TeslaMotors,hqqtbh2,"If help is needed, use our stickied support th...",1.640994e+09,t3_rt6fwg,2022-01-01 00:46:50,"AnalyzerOutput(output=NEU, probas={NEU: 0.832,...","[(NEG, 0.0038293171674013138), (NEU, 0.8324310...",0.159910
1,TeslaMotors,hqqtz7i,Requirements: \n\n• Fits under the aero cover ...,1.640995e+09,t3_rt6j33,2022-01-01 00:51:46,"AnalyzerOutput(output=NEU, probas={NEU: 0.872,...","[(NEG, 0.11134149879217148), (NEU, 0.871991336...",-0.094674
2,TeslaMotors,hqqu7z3,Photo credit @rkaplan1 on Twitter,1.640995e+09,t3_rt6kex,2022-01-01 00:53:37,"AnalyzerOutput(output=NEU, probas={NEU: 0.922,...","[(NEG, 0.003630931256338954), (NEU, 0.92183905...",0.070899
3,TeslaMotors,hqqxko8,I've wanted an Apple MagSafe charger in my cen...,1.640996e+09,t3_rt72vs,2022-01-01 01:20:13,"AnalyzerOutput(output=NEG, probas={NEG: 0.785,...","[(NEG, 0.784606397151947), (NEU, 0.20615413784...",-0.775367
4,TeslaMotors,hqr8kka,I see you set your temperature to: NICE.,1.641002e+09,t3_rt8o5j,2022-01-01 02:47:11,"AnalyzerOutput(output=POS, probas={POS: 0.675,...","[(NEG, 0.00296275457367301), (NEU, 0.322249382...",0.671825
...,...,...,...,...,...,...,...,...,...
168121,TeslaMotors,ie4b81w,Is this with the unreleased beta software? I’m...,1.656459e+09,t3_vn0xq2,2022-06-29 01:24:55,"AnalyzerOutput(output=NEG, probas={NEG: 0.778,...","[(NEG, 0.7782235741615295), (NEU, 0.2154842764...",-0.771931
168122,TeslaMotors,ie4bcyt,No still 10.12.2. Just in another vehicle.,1.656459e+09,t3_vn0yi6,2022-06-29 01:25:58,"AnalyzerOutput(output=NEU, probas={NEU: 0.973,...","[(NEG, 0.015009327791631222), (NEU, 0.97261792...",-0.002637
168123,TeslaMotors,ie4betk,"Also wondering this question, would love to re...",1.656459e+09,t3_vn0ysw,2022-06-29 01:26:22,"AnalyzerOutput(output=NEU, probas={NEU: 0.719,...","[(NEG, 0.0524548701941967), (NEU, 0.7192255258...",0.175865
168124,TeslaMotors,ie4nqju,Looks like a bunch of companies are all releas...,1.656465e+09,t3_vn0ysw,2022-06-29 03:03:36,"AnalyzerOutput(output=NEU, probas={NEU: 0.903,...","[(NEG, 0.005526799242943525), (NEU, 0.90288501...",0.086061


In [ ]:
#This is done to save the output of the analyzeroutput, so that above code does not have to be run again
df.to_csv('Tesla_Comments_Filtered_Analyzed.csv', index=False)

# Aggregation 

## Importing libraries

In [ ]:
import pandas as pd 
import numpy as np
import datetime
import pytz

## Defining the datasets

In [16]:
df1 = pd.read_csv('Tesla_Comments_Filtered_Analyzed.csv', delimiter=",")


In [20]:
TSLA = pd.read_csv('TSLA_2022.csv', delimiter=',')

In [21]:
TSLA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       250 non-null    object 
 1   Open       250 non-null    float64
 2   High       250 non-null    float64
 3   Low        250 non-null    float64
 4   Close      250 non-null    float64
 5   Adj Close  250 non-null    float64
 6   Volume     250 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


## Normalization of dates

In [22]:
# Converting it into datetime
TSLA['Date'] = pd.to_datetime(TSLA['Date'])


In [23]:
TSLA['date'] = TSLA['Date']

In [24]:
df1

,subreddit,id,body,created_utc,parent_id,date,emotion,probas,reg_output
0,TeslaMotors,hqqtbh2,"If help is needed, use our stickied support th...",1.640994e+09,t3_rt6fwg,2022-01-01 00:46:50,"AnalyzerOutput(output=NEU, probas={NEU: 0.832,...","[('NEG', 0.0038293171674013138), ('NEU', 0.832...",0.159910
1,TeslaMotors,hqqtz7i,Requirements: \n\n• Fits under the aero cover ...,1.640995e+09,t3_rt6j33,2022-01-01 00:51:46,"AnalyzerOutput(output=NEU, probas={NEU: 0.872,...","[('NEG', 0.11134149879217148), ('NEU', 0.87199...",-0.094674
2,TeslaMotors,hqqu7z3,Photo credit @rkaplan1 on Twitter,1.640995e+09,t3_rt6kex,2022-01-01 00:53:37,"AnalyzerOutput(output=NEU, probas={NEU: 0.922,...","[('NEG', 0.003630931256338954), ('NEU', 0.9218...",0.070899
3,TeslaMotors,hqqxko8,I've wanted an Apple MagSafe charger in my cen...,1.640996e+09,t3_rt72vs,2022-01-01 01:20:13,"AnalyzerOutput(output=NEG, probas={NEG: 0.785,...","[('NEG', 0.784606397151947), ('NEU', 0.2061541...",-0.775367
4,TeslaMotors,hqr8kka,I see you set your temperature to: NICE.,1.641002e+09,t3_rt8o5j,2022-01-01 02:47:11,"AnalyzerOutput(output=POS, probas={POS: 0.675,...","[('NEG', 0.00296275457367301), ('NEU', 0.32224...",0.671825
...,...,...,...,...,...,...,...,...,...
165105,TeslaMotors,ie4b81w,Is this with the unreleased beta software? I’m...,1.656459e+09,t3_vn0xq2,2022-06-29 01:24:55,"AnalyzerOutput(output=NEG, probas={NEG: 0.778,...","[('NEG', 0.7782235741615295), ('NEU', 0.215484...",-0.771931
165106,TeslaMotors,ie4bcyt,No still 10.12.2. Just in another vehicle.,1.656459e+09,t3_vn0yi6,2022-06-29 01:25:58,"AnalyzerOutput(output=NEU, probas={NEU: 0.973,...","[('NEG', 0.015009327791631222), ('NEU', 0.9726...",-0.002637
165107,TeslaMotors,ie4betk,"Also wondering this question, would love to re...",1.656459e+09,t3_vn0ysw,2022-06-29 01:26:22,"AnalyzerOutput(output=NEU, probas={NEU: 0.719,...","[('NEG', 0.0524548701941967), ('NEU', 0.719225...",0.175865
165108,TeslaMotors,ie4nqju,Looks like a bunch of companies are all releas...,1.656465e+09,t3_vn0ysw,2022-06-29 03:03:36,"AnalyzerOutput(output=NEU, probas={NEU: 0.903,...","[('NEG', 0.005526799242943525), ('NEU', 0.9028...",0.086061


In [27]:
df1['date'] = pd.to_datetime(df1['date'])

In [28]:
#Normalized the dates, so that it fits with the garch model. So that we do not have information spillover to the day we are trying to forecast.
df1['date'] = df1['date'].apply(lambda x: x.date() if x.time() < pd.Timestamp('14:30:00').time() else x.date() + pd.Timedelta(days=1))

In [31]:
df1['date'] = pd.to_datetime(df1['date'])

## Aggregation method

In [32]:
# group the data by day and calculate the average of 'reg_output' scores
df1 = df1.groupby(pd.Grouper(key='date', freq='D')).mean().reset_index()

C:\Users\maart\AppData\Local\Temp\ipykernel_9600\3007577693.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1 = df1.groupby(pd.Grouper(key='date', freq='D')).mean().reset_index()


In [33]:
#ASSUMPTION IS THAT IF THERE ARE NO COMMENTS, THEY ARE NEUTRAL. SO WE ASSUME IT HAS 0 IMPACT  
#For some of the days we dont have comments, so we convert those NAN's into 0's, since we cannot calculate with NANS, since it has no numerical value. 
df1 = df1.fillna(0)

## Merging the two datasets as one

In [34]:
#Here we make sure that the number of reddit scores match with the number of stock prices, so that we have the same amount of frequencies 
merged_df1 = pd.merge(TSLA, df1, on='date', how='inner')

In [35]:
merged_df1

,Date,Open,High,Low,Close,Adj Close,Volume,date,created_utc,reg_output
0,2022-02-28,271.670013,292.286682,271.570007,290.143341,290.143341,99006900,2022-02-28,1.646010e+09,-0.079532
1,2022-03-01,289.893341,296.626678,284.593323,288.123322,288.123322,74766900,2022-03-01,1.646087e+09,-0.086247
2,2022-03-02,290.709991,295.493347,281.423340,293.296661,293.296661,74643300,2022-03-02,1.646174e+09,-0.142537
3,2022-03-03,292.923340,295.480011,277.533325,279.763336,279.763336,61623600,2022-03-03,1.646276e+09,-0.088349
4,2022-03-04,283.033325,285.216675,275.053345,279.429993,279.429993,66999600,2022-03-04,1.646343e+09,-0.047891
...,...,...,...,...,...,...,...,...,...,...
240,2023-02-10,202.229996,206.199997,192.889999,196.889999,196.889999,204193800,2023-02-10,1.676015e+09,-0.369145
241,2023-02-13,194.419998,196.300003,187.610001,194.639999,194.639999,172475500,2023-02-13,1.676229e+09,-0.608041
242,2023-02-14,191.940002,209.820007,189.440002,209.250000,209.250000,216455700,2023-02-14,1.676336e+09,0.990002
243,2023-02-15,211.759995,214.660004,206.110001,214.240005,214.240005,182108600,2023-02-15,0.000000e+00,0.000000


In [36]:
merged_df1.isna().sum()

Date           0
Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
date           0
created_utc    0
reg_output     0
dtype: int64

In [37]:
merged_df1.to_csv('SP_SC.csv', index=False)